In [1]:
%load_ext autoreload
%autoreload 2


In [2]:

import tensorflow as tf
import numpy as np
import experiment_manager as em
import far_ho as far
from far_ho import hypergradients, hyperparameters

Data folder is C:\Users\lfranceschi\DATASETS
Experiment save directory is  C:\Users\lfranceschi\EXPERIMENTS


In [3]:
far.utils.check()

3


## single instance: 1 outer 1 inner problem (most common case I guess...) 
use `HyperOptimizer.minimize`

In [50]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])

v2 = tf.Variable([[-1., -2], [1., 0.]])
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape())

cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)
oo = tf.reduce_mean(v1*v2)

In [51]:
hyper_training_step = far.HyperOptimizer().minimize(oo, tf.train.AdamOptimizer(), 
                                                   cost, far.MomentumOptimizer(far.get_hyperparameter('lr', 0.1), 
                                                                              far.get_hyperparameter('mu', 0.9)))


In [52]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998]

In [53]:
hyper_training_step(10, _skip_hyper_ts=False)

ss.run(far.utils.hypergradients() + [oo])

[array([[ 0.2233436 ,  1.34006166],
        [-0.2233436 ,  0.        ]], dtype=float32),
 13.395198,
 0.28557462,
 0.0038858911]

In [55]:
far.utils.hypergradients()

[<tf.Variable 'lambda/hypergradient:0' shape=(2, 2) dtype=float32_ref>,
 <tf.Variable 'lr/hypergradient:0' shape=() dtype=float32_ref>,
 <tf.Variable 'mu/hypergradient:0' shape=() dtype=float32_ref>]

# TRY MULTIPLE DYNAMICS

In [44]:
try: ss.close()
except: pass
tf.reset_default_graph()
ss = tf.InteractiveSession()
v1 = tf.Variable([1.,3])
v2 = tf.Variable([[-1., -2], [1., 0.]])
lmbd = far.get_hyperparameter('lambda', 
                              initializer=tf.ones_initializer,
                             shape=v2.get_shape())
cost = tf.reduce_mean(v1**2) + tf.reduce_sum(lmbd*v2**2)

same = False
if same:
    v11 = tf.Variable([1.,3])
    v21 = tf.Variable([[-1., -2], [1., 0.]])
else:
    v11 = tf.Variable([13.,3])
    v21 = tf.Variable([[-10., -2], [1., -1.]])
cost1 = tf.reduce_mean(v11**2) + tf.reduce_sum(lmbd*v21**2)

io_optim = far.MomentumOptimizer(far.get_hyperparameter('eta', 0.1), 
                                   far.get_hyperparameter('mu', .9))

oo = tf.reduce_mean(v1*v2)
oo1 = tf.reduce_mean(v11*v21)

In [45]:
oo_optim = tf.train.GradientDescentOptimizer(0.0001)

farho = far.HyperOptimizer()
optim_dict = farho.set_dynamics(cost, io_optim, var_list=[v1, v2])
farho.set_problem(oo, optim_dict, oo_optim)
optim_dict = farho.set_dynamics(cost1, io_optim, var_list=[v11, v21])
farho.set_problem(oo1, optim_dict, oo_optim)
farho.finalize()



In [46]:
tf.global_variables_initializer().run()
ss.run(far.utils.hyperparameters())

[array([[ 1.,  1.],
        [ 1.,  1.]], dtype=float32), 0.1, 0.89999998]

In [47]:
farho.run(10, _skip_hyper_ts=False)

ss.run(far.utils.hypergradients() + [oo])

[array([[ 14.62900543,   1.34006166],
        [ -1.56340539,   0.33501542]], dtype=float32),
 147.34718,
 3.1413214,
 0.0038858911]

In [48]:
ss.run(far.utils.hyperparameters())

[array([[ 0.99853712,  0.99986601],
        [ 1.00015628,  0.9999665 ]], dtype=float32), 0.085265286, 0.89968586]

In [49]:
far.utils.hypergradients()

[<tf.Tensor 'lambda/hypergradient_2/Mean:0' shape=(2, 2) dtype=float32>,
 <tf.Tensor 'eta/hypergradient_2/Mean:0' shape=() dtype=float32>,
 <tf.Tensor 'mu/hypergradient_2/Mean:0' shape=() dtype=float32>]

In [79]:
# ss.run(ts_hy)  # perform an hypergradient descent step....
# print(ss.run(far.utils.hyperparameters()))
# print(ss.run(oo))